# Quantum Volume

The quantum volume is wholistic quantum computer performance measure [QVOL0, QVOL1, QVOL2].


Roughly the logarithm (base 2) of quantum volume $V_Q$, quantifies the largest random circuit of equal width and depth that the computer successfully implements and is certifiably quantum.

So if you have 64 qubits and the best log-QV you have measured is $\log_2V_Q = 3$ the effectively you only have an 8 qubit device!



The quantum volume metric accounts for all relevant hardware parameters, e.g.
* coherence
* calibration errors
* crosstalk 
* spectator errors
* gate fidelity, measurement fidelity, initialization fidelity
* qubit connectivity 
* gate set

### Some details

A circuit to measure the quantum volume consists of layers of  a permuations of $n$ qubits followed by nearest neighbour two qubit Haar random unitaries. At the end we measure in the computational basis. See Figure 1.

<figure>
  <img src="figs/q-vol-from-arxiv-paper.png" alt="Drawing" style="width: 350px;"/>
  <figcaption>
      Figure 1. from <a href="https://arxiv.org/abs/1811.12926">Cross et al. [QVOL0]</a>.
      A circuit to measure the quantum volume consists of a permuation gate on $n$ qubits denoted by $\pi$ in the above figure. The gate ${\rm SU}(4)$ denotes are two qubit gate drawn from the Haar measure on the <a href="https://en.wikipedia.org/wiki/Special_unitary_group">special unitary group of degree 4</a>.
  </figcaption>
</figure>


The certification of quantumness comes from the so called **Heavy Output Generation** problem and the **Quantum Threshold Assumption** see [HOG] for more details.


    [QVOL0] Validating quantum computers using randomized model circuits
            Cross et al.
            arXiv:1811.12926 (2018)
            https://arxiv.org/abs/1811.12926

    [QVOL1] Quantum optimization using variational algorithms on near-term quantum devices
            Moll et al.
            Quantum Science and Technology 3, Number 3 2018
            https://doi.org/10.1088/2058-9565/aab822
            https://arxiv.org/abs/1710.01022

    [QVOL2] Quantum Volume
            Bishop et al.
            (2017)
            https://ibm.co/2NdyRvf
        
    [HOG] Complexity-Theoretic Foundations of Quantum Supremacy Experiments
          Aaronson and Chen
          32nd Computational Complexity Conference, vol 79 (CCC 2017)
          https://doi.org/10.4230/LIPIcs.CCC.2017.22
          https://arxiv.org/abs/1612.05903

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

from pyquil.api import get_qc

ideal_qc = get_qc('4q-qvm', noisy=False)
noisy_qc = get_qc("4q-noisy-qvm", noisy=True)

qubits = ideal_qc.qubits()
print(qubits)
graph = ideal_qc.qubit_topology()
nx.draw_networkx(graph, with_labels=True)

## Import and setup logging for info on progress.

In [ ]:
from forest.benchmarking.quantum_volume import measure_quantum_volume

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
show_progress_bar = True

## Replicate FIG2. of [QVol]  (See forest_qcvv.quantum_volume for ref)

This is SLOW--it takes about 4 minutes.

In [ ]:
ideal_outcomes = measure_quantum_volume(ideal_qc, num_circuits=200, show_progress_bar=show_progress_bar)

## Now with noise

This is SLOW--it takes about 5 minutes, even with half the number of shots from above.

In [ ]:
noisy_outcomes = measure_quantum_volume(noisy_qc, num_circuits=200, num_shots=500, show_progress_bar=show_progress_bar)

In [ ]:
depths = np.arange(2, 5)
ideal_probs = [ideal_outcomes[depth][0] if depth in ideal_outcomes.keys() else 0 for depth in depths]
noisy_probs = [noisy_outcomes[depth][0] if depth in noisy_outcomes.keys() else 0 for depth in depths]

plt.axhline(.5 + np.log(2)/2, color='b', ls='--', label='ideal asymptote')
plt.axhline(2/3, color='black', ls=':', label='achievable threshold')
plt.scatter(np.array(depths) - .1, ideal_probs, color='b', label='ideal simulation')
plt.scatter(depths, noisy_probs, color='r', label='noisy simulation')
plt.ylabel("est. heavy output probability h_d")
plt.xlabel("width/depth of model circuit m=d")
plt.ylim(.4,1.0)
plt.xlim(1.8, 4.2)
plt.xticks(depths)
plt.legend(loc='lower left')
plt.show()

## Try a different topology

In [ ]:
n = 4
path_graph = nx.path_graph(n)
loop_graph = nx.cycle_graph(n)
four_pointed_star = nx.star_graph(n)

In [ ]:
nx.draw_networkx(path_graph, with_labels=True)

In [ ]:
nx.draw_networkx(loop_graph, with_labels=True)

In [ ]:
nx.draw_networkx(four_pointed_star, with_labels=True)

In [ ]:
from pyquil.api._quantum_computer import _get_qvm_with_topology
path_qc = _get_qvm_with_topology(name='path', topology=path_graph, noisy=True)

In [ ]:
path_outcomes = measure_quantum_volume(path_qc, num_circuits=200, num_shots=500, show_progress_bar=show_progress_bar)

## Compare to noisy complete topology

In [ ]:
depths = np.arange(2, 5)
ideal_probs = [ideal_outcomes[depth][0] if depth in ideal_outcomes.keys() else 0 for depth in depths]
noisy_probs = [noisy_outcomes[depth][0] if depth in noisy_outcomes.keys() else 0 for depth in depths]
path_probs = [path_outcomes[depth][0] if depth in path_outcomes.keys() else 0 for depth in depths]

plt.axhline(.5 + np.log(2)/2, color='b', ls='--', label='ideal asymptote')
plt.axhline(2/3, color='grey', ls=':', label='achievable threshold')
plt.scatter(np.asarray(depths) - .1, ideal_probs, color='b', label='ideal simulation')
plt.scatter(depths, noisy_probs, color='r', label='noisy complete simulation')
plt.scatter(np.asarray(depths) + .1, path_probs, color='black', label='noisy path simulation')
plt.ylabel("est. heavy output probability h_d")
plt.xlabel("width/depth of model circuit m=d")
plt.ylim(.4,1.0)
plt.xlim(1.8, 4.2)
plt.xticks(depths)
plt.legend(loc='lower left')
plt.show()

## Run intermediate steps yourself

In [ ]:
from forest.benchmarking.quantum_volume import generate_abstract_qv_circuit, _naive_program_generator, collect_heavy_outputs
from pyquil.numpy_simulator import NumpyWavefunctionSimulator
from pyquil.gates import RESET
import time

def generate_circuits(depths):
    for d in depths:
        yield generate_abstract_qv_circuit(d)

def convert_ckts_to_programs(qc, circuits, qubits=None):
    for idx, ckt in enumerate(circuits):
        if qubits is None:
            d_qubits = qc.qubits()  # by default the program can act on any qubit in the computer
        else:
            d_qubits = qubits[idx]

        yield _naive_program_generator(qc, d_qubits, *ckt)


def acquire_quantum_volume_data(qc, programs, num_shots = 1000, use_active_reset = False):
    for program in programs:
        start = time.time()

        if use_active_reset:
            reset_measure_program = Program(RESET())
            program = reset_measure_program + program

        # run the program num_shots many times
        program.wrap_in_numshots_loop(num_shots)
        executable = qc.compiler.native_quil_to_executable(program)

        results = qc.run(executable)

        runtime = time.time() - start
        yield results


def acquire_heavy_hitters(abstract_circuits):
    for ckt in abstract_circuits:
        perms, gates = ckt
        depth = len(perms)
        wfn_sim = NumpyWavefunctionSimulator(depth)

        start = time.time()
        heavy_outputs = collect_heavy_outputs(wfn_sim, perms, gates)
        runtime = time.time() - start

        yield heavy_outputs


Generate (len(unique_depths) x n_circuits) many "Abstract Ckt"s that describe each model circuit for each depth.

In [ ]:
n_circuits = 100
unique_depths = [2,3]
depths = [d for d in unique_depths for _ in range(n_circuits)]
ckts = list(generate_circuits(depths))
print(ckts[0])

Use the _naive_program_generator to synthesize native pyquil programs that implement each ckt natively on the qc.

In [ ]:
progs = list(convert_ckts_to_programs(noisy_qc, ckts))
print(progs[0])

Run the programs. This can be slow.

In [ ]:
num_shots=10
results = list(acquire_quantum_volume_data(noisy_qc, progs, num_shots=num_shots))

Classically simulate the circuits to get heavy hitters, and record how many hh were sampled for each program run.

In [ ]:
ckt_hhs = acquire_heavy_hitters(ckts)

Count the number of heavy hitters that were sampled on the qc

In [ ]:
from forest.benchmarking.quantum_volume import count_heavy_hitters_sampled
num_hh_sampled = count_heavy_hitters_sampled(results, ckt_hhs)

Get estimates of the probability of sampling hh at each depth, and the lowerbound on that estimate

In [ ]:
from forest.benchmarking.quantum_volume import get_prob_sample_heavy_by_depth

results = get_prob_sample_heavy_by_depth(depths, num_hh_sampled, [num_shots for _ in depths])
results

Use the results to get a lower bound on the quantum volume

In [ ]:
from forest.benchmarking.quantum_volume import extract_quantum_volume_from_results
qv = extract_quantum_volume_from_results(results)
qv